In [47]:
# import pandas_ta as ta
import pandas as pd
import pandas_ta as ta
import numpy as np
from datetime import datetime, timedelta
# import plotly.graph_objects as go
import warnings
# from binance.client import Client   # Load historical price data from binance exchange 
# client = Client()
warnings.filterwarnings("ignore", category=FutureWarning)
import asyncio
from binance import AsyncClient
import time
import myUtils
from botConfig import *
# from myUtils import *
pd.set_option('display.max_rows', 250)

In [48]:
import nest_asyncio
nest_asyncio.apply()

In [49]:
import importlib
# If you want to update the module:
importlib.reload(myUtils)
from myUtils import *

In [50]:
cb = ccxtBinance()

await cb.binanceActivate(mode)

starting = time.time()
qty = await cb.getqty('USDT')
print(qty)

Authentication Successful.
Exchange closed.
30043.73397668


In [5]:
coins = pd.read_csv('spot_coins_list.csv').loc[:,'0'].tolist()
coins

['BTCUSDT',
 'ETHUSDT',
 'BNBUSDT',
 'BCCUSDT',
 'NEOUSDT',
 'LTCUSDT',
 'QTUMUSDT',
 'ADAUSDT',
 'XRPUSDT',
 'EOSUSDT',
 'TUSDUSDT',
 'IOTAUSDT',
 'XLMUSDT',
 'ONTUSDT',
 'TRXUSDT',
 'ETCUSDT',
 'ICXUSDT',
 'VENUSDT',
 'NULSUSDT',
 'VETUSDT',
 'PAXUSDT',
 'BCHABCUSDT',
 'BCHSVUSDT',
 'USDCUSDT',
 'LINKUSDT',
 'WAVESUSDT',
 'BTTUSDT',
 'USDSUSDT',
 'ONGUSDT',
 'HOTUSDT',
 'ZILUSDT',
 'ZRXUSDT',
 'FETUSDT',
 'BATUSDT',
 'XMRUSDT',
 'ZECUSDT',
 'IOSTUSDT',
 'CELRUSDT',
 'DASHUSDT',
 'NANOUSDT',
 'OMGUSDT',
 'THETAUSDT',
 'ENJUSDT',
 'MITHUSDT',
 'MATICUSDT',
 'ATOMUSDT',
 'TFUELUSDT',
 'ONEUSDT',
 'FTMUSDT',
 'ALGOUSDT',
 'USDSBUSDT',
 'GTOUSDT',
 'ERDUSDT',
 'DOGEUSDT',
 'DUSKUSDT',
 'ANKRUSDT',
 'WINUSDT',
 'COSUSDT',
 'NPXSUSDT',
 'COCOSUSDT',
 'MTLUSDT',
 'TOMOUSDT',
 'PERLUSDT',
 'DENTUSDT',
 'MFTUSDT',
 'KEYUSDT',
 'STORMUSDT',
 'DOCKUSDT',
 'WANUSDT',
 'FUNUSDT',
 'CVCUSDT',
 'CHZUSDT',
 'BANDUSDT',
 'BUSDUSDT',
 'BEAMUSDT',
 'XTZUSDT',
 'RENUSDT',
 'RVNUSDT',
 'HCUSDT',
 'HBARUSD

In [6]:
timeframe = "1m"
start = "2023-12-01"
# start_date = datetime.strptime(start, "%Y-%m-%d")
end = str(datetime.now())
# Calculate the difference in days
# day = (datetime.now() - start_date).days

# OTHER PARAMETERS
fast_ema_period=9
slow_ema_period=18

usdt_amount = 10 # usdt amount allocated for each bot

tp_perc= 10

In [34]:
import nest_asyncio
nest_asyncio.apply()

starting = time.time()

signals_dict = {'coins' : [], 'buy_signals' : [], 'sell_signals' : []}
async def fetch_ohlcv(coin):
    try:
        data = await getdata(coin, timeframe)
        print(coin, data)
        signals_dict['coins'].append(coin)
        signals_dict['buy_signals'].append(data.buy_signal.iloc[-2])
        signals_dict['sell_signals'].append(data.sell_signal.iloc[-2])
    except Exception as e:
        print(coin, e)
        return None
    return data

async def fetch_all_ohlcv(coins):

    tasks = [fetch_ohlcv(coin) for coin in coins]
    results = await asyncio.gather(*tasks)

    return results

async def main():
    ohlcv_data = []
    client = await AsyncClient.create()
    ohlcv_data = await fetch_all_ohlcv(coins)
    await client.close_connection()
    return ohlcv_data

data = await main()
    
ending = time.time()

total_time = ending - starting
print("It took {} seconds to make {} API calls".format(total_time, len(coins)))

GTOUSDT                         Open     High      Low    Close    Volume  fast_ema  \
timestamp                                                                     
2022-11-28 02:40:00  0.01241  0.01241  0.01223  0.01230  757016.4       NaN   
2022-11-28 02:41:00  0.01230  0.01230  0.01230  0.01230   15710.7       NaN   
2022-11-28 02:42:00  0.01230  0.01230  0.01230  0.01230       0.0       NaN   
2022-11-28 02:43:00  0.01230  0.01230  0.01229  0.01229   47668.4       NaN   
2022-11-28 02:44:00  0.01228  0.01236  0.01228  0.01236  134480.5       NaN   
2022-11-28 02:45:00  0.01233  0.01233  0.01233  0.01233    1213.6       NaN   
2022-11-28 02:46:00  0.01233  0.01234  0.01233  0.01234   14845.7       NaN   
2022-11-28 02:47:00  0.01234  0.01234  0.01234  0.01234    3423.8       NaN   
2022-11-28 02:48:00  0.01236  0.01237  0.01236  0.01237    9867.6  0.012326   
2022-11-28 02:49:00  0.01234  0.01234  0.01233  0.01234    2578.9  0.012328   
2022-11-28 02:50:00  0.01234  0.01234  0.012

In [35]:
signals_df = pd.DataFrame(signals_dict).set_index('coins')
signals_df

,buy_signals,sell_signals
coins,,
GTOUSDT,False,False
TRXUSDT,False,False
BNBUSDT,False,False
NPXSUSDT,False,False
LINKUSDT,False,False
FETUSDT,False,False
WAVESUSDT,False,False
ICXUSDT,False,False
DOGEUSDT,False,False


In [36]:
buysignals = signals_df[signals_df['buy_signals'] == True].index.to_list()
sellsignals = signals_df[signals_df['sell_signals'] == True].index.to_list()
buysignals

['PAXUSDT', 'STPTUSDT']

In [37]:
sellsignals

[]

## BOT

In [38]:
# this should be saved in json or 
running_bots = 0
max_running_bots = 10

In [72]:
in_position = {coin: False for coin in coins} # intialize the in_position dict for the first time

In [73]:
in_position

{'BTCUSDT': False,
 'ETHUSDT': False,
 'BNBUSDT': False,
 'BCCUSDT': False,
 'NEOUSDT': False,
 'LTCUSDT': False,
 'QTUMUSDT': False,
 'ADAUSDT': False,
 'XRPUSDT': False,
 'EOSUSDT': False,
 'TUSDUSDT': False,
 'IOTAUSDT': False,
 'XLMUSDT': False,
 'ONTUSDT': False,
 'TRXUSDT': False,
 'ETCUSDT': False,
 'ICXUSDT': False,
 'VENUSDT': False,
 'NULSUSDT': False,
 'VETUSDT': False,
 'PAXUSDT': False,
 'BCHABCUSDT': False,
 'BCHSVUSDT': False,
 'USDCUSDT': False,
 'LINKUSDT': False,
 'WAVESUSDT': False,
 'BTTUSDT': False,
 'USDSUSDT': False,
 'ONGUSDT': False,
 'HOTUSDT': False,
 'ZILUSDT': False,
 'ZRXUSDT': False,
 'FETUSDT': False,
 'BATUSDT': False,
 'XMRUSDT': False,
 'ZECUSDT': False,
 'IOSTUSDT': False,
 'CELRUSDT': False,
 'DASHUSDT': False,
 'NANOUSDT': False,
 'OMGUSDT': False,
 'THETAUSDT': False,
 'ENJUSDT': False,
 'MITHUSDT': False,
 'MATICUSDT': False,
 'ATOMUSDT': False,
 'TFUELUSDT': False,
 'ONEUSDT': False,
 'FTMUSDT': False,
 'ALGOUSDT': False,
 'USDSBUSDT': False,
 '

In [74]:
trades_dict = {'buy_order_data' : [], 'sell_order_data' : []} # initialize for the first time after that we need 
#                                                                     to just update this

In [80]:
async def buyCoin(coin):
    try:
        orderSize = await cb.calculate_order_size(coin, usdt_amount)
        print(coin, orderSize)
        order = await cb.place_buy_order(coin, orderSize)
        print(order)

        # Append buy order information to trades_dict
        trades_dict['buy_order_data'].append(
            {'coin': coin, 'order': order, 'order_size': orderSize})

        return order
    except Exception as e:
        print(coin, e)
        return None


async def sellCoin(coin):
    try:
        orderSize = await cb.getqty(coin[:-4])
        print(coin, orderSize)
        order = await cb.place_sell_order(coin, orderSize)
        print(order)

        # Append sell order information to trades_dict
        trades_dict['sell_order_data'].append(
            {'coin': coin, 'order': order, 'order_size': orderSize})

        return order
    except Exception as e:
        print(coin, e)
        return None

In [81]:
try:
    if len(buysignals) > 0:
        for coin in buysignals:
            print(coin)
            if running_bots <= max_running_bots:
                print(running_bots, max_running_bots)
                if not in_position[coin]:
                    print('buy please')
                    # Use 'await' for asynchronous function
                    order = await buyCoin(coin)
                    print(order)
                    # trades_dict['buy_order_data'].append(order)
                    running_bots += 1

    if len(sellsignals) > 0:
        for coin in sellsignals:
            if in_position[coin]:
                # Use 'await' for asynchronous function
                order = await sellCoin(coin)
                # trades_dict['sell_order_data'].append(order)
                running_bots -= 1
except Exception as e:
    print(e)

PAXUSDT
4 10
buy please
Error calculating order size for PAXUSDT: binance does not have market symbol PAXUSDT
PAXUSDT None
Error placing buy order for PAXUSDT: binance does not have market symbol PAXUSDT
False
False
STPTUSDT
5 10


In [76]:
async def main():
    balances = await cb.in_pos('BTC') # this is just to access all the qty values of different coins after first run
                                            # this should run at the end of the bot to update positions
        

    # update the in_position dict, after first run this should run at the end of the code
    for coin in coins:
        coin = coin[:-4]
        try:
            asset = float([i['free'] for i in balances[1] if i['asset'] == coin][0])
            if asset > 0:
                in_position[coin+'USDT'] = True
            else:
                in_position[coin+'USDT'] = False
        except:
            None

# Run the event loop
asyncio.run(main())

In [79]:
in_position

{'BTCUSDT': True,
 'ETHUSDT': True,
 'BNBUSDT': True,
 'BCCUSDT': False,
 'NEOUSDT': True,
 'LTCUSDT': True,
 'QTUMUSDT': True,
 'ADAUSDT': True,
 'XRPUSDT': True,
 'EOSUSDT': True,
 'TUSDUSDT': True,
 'IOTAUSDT': True,
 'XLMUSDT': True,
 'ONTUSDT': True,
 'TRXUSDT': True,
 'ETCUSDT': True,
 'ICXUSDT': True,
 'VENUSDT': False,
 'NULSUSDT': True,
 'VETUSDT': True,
 'PAXUSDT': False,
 'BCHABCUSDT': False,
 'BCHSVUSDT': False,
 'USDCUSDT': False,
 'LINKUSDT': True,
 'WAVESUSDT': True,
 'BTTUSDT': False,
 'USDSUSDT': False,
 'ONGUSDT': True,
 'HOTUSDT': True,
 'ZILUSDT': True,
 'ZRXUSDT': True,
 'FETUSDT': True,
 'BATUSDT': True,
 'XMRUSDT': True,
 'ZECUSDT': True,
 'IOSTUSDT': True,
 'CELRUSDT': True,
 'DASHUSDT': True,
 'NANOUSDT': False,
 'OMGUSDT': True,
 'THETAUSDT': True,
 'ENJUSDT': True,
 'MITHUSDT': False,
 'MATICUSDT': True,
 'ATOMUSDT': True,
 'TFUELUSDT': True,
 'ONEUSDT': True,
 'FTMUSDT': True,
 'ALGOUSDT': True,
 'USDSBUSDT': False,
 'GTOUSDT': False,
 'ERDUSDT': False,
 'DO

In [77]:
balances[1]

[{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'},
 {'asset': 'BTC', 'free': '0.72255000', 'locked': '0.00000000'},
 {'asset': 'LTC', 'free': '6.00000000', 'locked': '0.00000000'},
 {'asset': 'BNB', 'free': '2.00000000', 'locked': '0.00000000'},
 {'asset': 'USDT', 'free': '30043.73397668', 'locked': '0.00000000'},
 {'asset': 'TRX', 'free': '4758.00000000', 'locked': '0.00000000'},
 {'asset': 'XRP', 'free': '832.00000000', 'locked': '0.00000000'},
 {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
 {'asset': 'NEO', 'free': '42.00000000', 'locked': '0.00000000'},
 {'asset': 'QTUM', 'free': '155.00000000', 'locked': '0.00000000'},
 {'asset': 'EOS', 'free': '677.00000000', 'locked': '0.00000000'},
 {'asset': 'SNT', 'free': '10979.00000000', 'locked': '0.00000000'},
 {'asset': 'BNT', 'free': '645.00000000', 'locked': '0.00000000'},
 {'asset': 'GAS', 'free': '66.00000000', 'locked': '0.00000000'},
 {'asset': 'WTC', 'free': '13665.00000000', 'locked': '0.00000

In [78]:
trades_dict

{'buy_order_data': [{'coin': 'PAXUSDT', 'order': False, 'order_size': None},
  {'coin': 'STPTUSDT',
   'order': {'info': {'symbol': 'STPTUSDT',
     'orderId': '1377782',
     'orderListId': '-1',
     'clientOrderId': 'x-R4BD3S825dac41fd6ff5fe391a6b0c',
     'transactTime': '1704368056672',
     'price': '0.00000000',
     'origQty': '144.40000000',
     'executedQty': '144.40000000',
     'cummulativeQuoteQty': '9.99248000',
     'status': 'FILLED',
     'timeInForce': 'GTC',
     'type': 'MARKET',
     'side': 'BUY',
     'workingTime': '1704368056672',
     'fills': [{'price': '0.06920000',
       'qty': '144.40000000',
       'commission': '0.00000000',
       'commissionAsset': 'STPT',
       'tradeId': '40150'}],
     'selfTradePreventionMode': 'EXPIRE_MAKER'},
    'id': '1377782',
    'clientOrderId': 'x-R4BD3S825dac41fd6ff5fe391a6b0c',
    'timestamp': 1704368056672,
    'datetime': '2024-01-04T11:34:16.672Z',
    'lastTradeTimestamp': 1704368056672,
    'lastUpdateTimestamp':

In [62]:
for coin in buysignals:
    await buyCoin(coin)

Error calculating order size for PAXUSDT: binance does not have market symbol PAXUSDT
PAXUSDT None
Error placing buy order for PAXUSDT: binance does not have market symbol PAXUSDT
False
STPTUSDT 144.4460494005489
{'info': {'symbol': 'STPTUSDT', 'orderId': '1372768', 'orderListId': '-1', 'clientOrderId': 'x-R4BD3S82284120f60826c1ec919f8e', 'transactTime': '1704364659428', 'price': '0.00000000', 'origQty': '144.40000000', 'executedQty': '144.40000000', 'cummulativeQuoteQty': '10.00836400', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': '1704364659428', 'fills': [{'price': '0.06931000', 'qty': '144.40000000', 'commission': '0.00000000', 'commissionAsset': 'STPT', 'tradeId': '39702'}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}, 'id': '1372768', 'clientOrderId': 'x-R4BD3S82284120f60826c1ec919f8e', 'timestamp': 1704364659428, 'datetime': '2024-01-04T10:37:39.428Z', 'lastTradeTimestamp': 1704364659428, 'lastUpdateTimestamp': 1704364659428, 'symbol':

In [63]:
await sellCoin('BIFIUSDT')

BIFIUSDT 0.027
{'info': {'symbol': 'BIFIUSDT', 'orderId': '872454', 'orderListId': '-1', 'clientOrderId': 'x-R4BD3S827b5a883fe0ebc2ce3834cc', 'transactTime': '1704364689815', 'price': '0.00000000', 'origQty': '0.02700000', 'executedQty': '0.02700000', 'cummulativeQuoteQty': '9.99000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'workingTime': '1704364689815', 'fills': [{'price': '370.00000000', 'qty': '0.02700000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'tradeId': '7663'}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}, 'id': '872454', 'clientOrderId': 'x-R4BD3S827b5a883fe0ebc2ce3834cc', 'timestamp': 1704364689815, 'datetime': '2024-01-04T10:38:09.815Z', 'lastTradeTimestamp': 1704364689815, 'lastUpdateTimestamp': 1704364689815, 'symbol': 'BIFI/USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'sell', 'price': 370.0, 'triggerPrice': None, 'amount': 0.027, 'cost': 9.99, 'average': 370.0, 'filled':

{'info': {'symbol': 'BIFIUSDT',
  'orderId': '872454',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S827b5a883fe0ebc2ce3834cc',
  'transactTime': '1704364689815',
  'price': '0.00000000',
  'origQty': '0.02700000',
  'executedQty': '0.02700000',
  'cummulativeQuoteQty': '9.99000000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'workingTime': '1704364689815',
  'fills': [{'price': '370.00000000',
    'qty': '0.02700000',
    'commission': '0.00000000',
    'commissionAsset': 'USDT',
    'tradeId': '7663'}],
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 'id': '872454',
 'clientOrderId': 'x-R4BD3S827b5a883fe0ebc2ce3834cc',
 'timestamp': 1704364689815,
 'datetime': '2024-01-04T10:38:09.815Z',
 'lastTradeTimestamp': 1704364689815,
 'lastUpdateTimestamp': 1704364689815,
 'symbol': 'BIFI/USDT',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'reduceOnly': None,
 'side': 'sell',
 'price': 370.0,
 'triggerPrice': None,
 'amount': 0.

## BOT OLD

In [64]:
try:
    order = await cb.place_sell_order(coins[0], 0.0002)
    print(order)
except Exception as e:
    print(f"Error in example_function: {e}")

{'info': {'symbol': 'BTCUSDT', 'orderId': '16853196', 'orderListId': '-1', 'clientOrderId': 'x-R4BD3S824baa26550cd51a341ee7d2', 'transactTime': '1704365346838', 'price': '0.00000000', 'origQty': '0.00020000', 'executedQty': '0.00020000', 'cummulativeQuoteQty': '8.61160000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'workingTime': '1704365346838', 'fills': [{'price': '43058.00000000', 'qty': '0.00020000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'tradeId': '2847888'}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}, 'id': '16853196', 'clientOrderId': 'x-R4BD3S824baa26550cd51a341ee7d2', 'timestamp': 1704365346838, 'datetime': '2024-01-04T10:49:06.838Z', 'lastTradeTimestamp': 1704365346838, 'lastUpdateTimestamp': 1704365346838, 'symbol': 'BTC/USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': None, 'side': 'sell', 'price': 43058.0, 'triggerPrice': None, 'amount': 0.0002, 'cost': 8.6116, 'average': 43058.0, 'filled': 

In [65]:
order_size = await cb.calculate_order_size(coins[0], usdt_amount)
order_size

0.00023227956611106168

In [66]:
import asyncio

starting = time.time()

trades_dict = {'coins': [], 'orderData': []}


async def buyCoin(coin):
    try:
        orderSize = await cb.calculate_order_size(coin, usdt_amount)
        print(coin, orderSize)
        data = await cb.place_buy_order(coin, orderSize)
        trades_dict['coins'].append(coin)
        trades_dict['orderData'].append(data)
        return data
    except Exception as e:
        print(coin, e)
        return None


async def buy_all_coins(buysignals):
    tasks = [asyncio.create_task(buyCoin(coin)) for coin in buysignals]
    results = await asyncio.gather(*tasks)
    await asyncio.sleep(1)  # Add a delay to prevent timestamp issues
    return results


async def main():
    ohlcv_data = []
    client = await AsyncClient.create()
    ohlcv_data = await buy_all_coins(buysignals)
    await client.close_connection()
    return ohlcv_data

# Use await in a Jupyter notebook cell
loop = asyncio.new_event_loop()
# Use await directly in a Jupyter notebook cell
await main()
await cb.close_exchange()

ending = time.time()

total_time = ending - starting
print("It took {} seconds to make {} API calls".format(
    total_time, len(buysignals)))

Error calculating order size for PAXUSDT: binance does not have market symbol PAXUSDT
PAXUSDT None
Error placing buy order for PAXUSDT: binance does not have market symbol PAXUSDT
STPTUSDT 140.76576576576576
Exchange closed.
It took 2.892488956451416 seconds to make 2 API calls


In [67]:
trades_dict

{'coins': ['PAXUSDT', 'STPTUSDT'],
 'orderData': [False,
  {'info': {'symbol': 'STPTUSDT',
    'orderId': '1373840',
    'orderListId': '-1',
    'clientOrderId': 'x-R4BD3S82622245f300d7480df8cd88',
    'transactTime': '1704365354421',
    'price': '0.00000000',
    'origQty': '140.70000000',
    'executedQty': '140.70000000',
    'cummulativeQuoteQty': '9.98125800',
    'status': 'FILLED',
    'timeInForce': 'GTC',
    'type': 'MARKET',
    'side': 'BUY',
    'workingTime': '1704365354421',
    'fills': [{'price': '0.07094000',
      'qty': '140.70000000',
      'commission': '0.00000000',
      'commissionAsset': 'STPT',
      'tradeId': '39845'}],
    'selfTradePreventionMode': 'EXPIRE_MAKER'},
   'id': '1373840',
   'clientOrderId': 'x-R4BD3S82622245f300d7480df8cd88',
   'timestamp': 1704365354421,
   'datetime': '2024-01-04T10:49:14.421Z',
   'lastTradeTimestamp': 1704365354421,
   'lastUpdateTimestamp': 1704365354421,
   'symbol': 'STPT/USDT',
   'type': 'market',
   'timeInForce